In [1]:
# set your working directory
import os
os.getcwd()
os.chdir('C:/Users/Mark/Documents/GitHub/eos_biomarkers/')

In [2]:
# import packages

# general math, stats, dataframe handling
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.stats.multitest import multipletests
import random

# plotting
import matplotlib.pyplot as plt
import seaborn as sns

# imputation, random forests things
from itertools import compress
from sklearn.decomposition import PCA
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score,confusion_matrix
import sklearn.metrics as metrics

import warnings
warnings.filterwarnings("ignore")

# data:
df = pd.read_csv("data/allbatchesclin.csv") # normalized ms data for each protein for each sample w/ log2 abundance
new_df = pd.read_csv('data/leena_proteins_mean2023_nu037fix.csv', index_col = 0) # reshaped dataframe w/ rows for each sample & cols for each protein + some clinical variables
# ^ this differs from the one generated in the code; correct sex info was manually added for nu037.
psmetadata = pd.read_csv('data/ps_metadata_cut.csv') # metadata for each sample
# this differs from ps_metadata.csv in that 1. it lacks chart notes and 2. BP484 & NU125 have been recoded to 1 for prom
mesoscale = pd.read_excel('data/cord_mesoscaledata_12_9_22.xlsx', engine='openpyxl')


In [3]:
#incorporating sex into the analysis
merge_df = df.loc[:,['sex','sample_id']]
merge_df = merge_df.drop_duplicates()
new_df['sample_id'] = new_df.index
new_df = pd.merge(new_df, merge_df, how = 'left', on = 'sample_id')


In [4]:
print(new_df[new_df['sample_id'] == 'NU037'])
# NU037 has data for sex, gestational age, etc, in ps_metadata.csv but not in allbatchesclin.csv or in the file that becomes new_df
# I manually copied the data from ps_metadata.csv into the file that becomes new_df, but it's too laborious to fix allbatchesclin
# so it needs manual removal since the discrepancy duplicates it.
new_df = new_df.drop(new_df[(new_df['sample_id'] == 'NU037') & (new_df['sex'].isna())].index, axis=0)

          ALB  HP;HPR   SERPINA1        A2M      IGHG1      IGHG2  IGHG3  \
95  31.732363     NaN  28.198313  27.536863  27.768313  22.924263    NaN   
96  31.732363     NaN  28.198313  27.536863  27.768313  22.924263    NaN   

        APOA1        FGA  FGB  ...  PSME2  CORO1C  CLIC4;CLIC2;CLIC5;CLIC6  \
95  31.524213  25.201763  NaN  ...    NaN     NaN                      NaN   
96  31.524213  25.201763  NaN  ...    NaN     NaN                      NaN   

    ga_cat  epoch  sample_type  GA_wks  batch  sample_id   sex  
95    >=37      4      Control   37.29      2      NU037   NaN  
96    >=37      4      Control   37.29      2      NU037  Male  

[2 rows x 444 columns]


In [5]:
#removing all the controls and presumed sepsis patients
#new_df = new_df[new_df['sample_type'] != 'PS_nonclassicEOS']
new_df = new_df[new_df['sample_type'] != 'nonclassicEOS']
new_df = new_df[new_df['sample_type'] != 'Control + degradation assessment sample']
new_df = new_df[new_df['sample_type'] != 'pooled_control']
#new_df = new_df[new_df['sample_type'] != 'PS']

In [6]:
#removing proteins that have over 80% missingness
tmp = new_df.isnull().sum()/new_df.shape[0]
new_df =  new_df.drop(tmp[tmp>0.80].index, axis = 1)

In [7]:
# imputing values for heatmap visualizations (which do not play nice with NAs)
# first: for proteins with less than 30% missingness, use an iterative imputer by sample type
# then: for proteins with greater missingness, just sample from the bottom 10% of values
# the rationale for this approach is that since most missing proteins arose from batch effects,
#   and the batches are 25% each, so proteins with >25-30% missingness are most likely truly non-abundant
#   rather than just not detected in a given batch.
new_df_imp = new_df.copy() #copy it
imputeddf = pd.DataFrame() # this is to save just the imputed things
imp = IterativeImputer(max_iter=10, random_state=0) #define iterative imputer funxion

#for st in ['EOS','Control','PS']:
for st in ['EOS','Control']: #do the following for each sample type:
    tmp = new_df[new_df['sample_type'] == st] #store in tmp only the rows in new_df that have the current sample type
    vals =  tmp.iloc[:, :-7] # grab all cols except the last seven! which are metadata.
    missing = vals.isnull().sum()/vals.shape[0] #for each column, what % are missing?
    keep = missing[missing<0.3].index # names of columns (proteins) where they mostly aren't missing.
    vals_imp = imp.fit_transform(vals.loc[:,keep]) # for the proteins in keep, impute
    vals_imp = pd.DataFrame(vals_imp, index = vals.index, columns = keep) #make it a dataframe
    new_df_imp.loc[vals_imp.index, vals_imp.columns] = vals_imp
    imputeddf = pd.concat([imputeddf, vals_imp]) # save just the imputed things
imputed_labels = new_df.loc[new_df_imp.index, 'sample_type']
imputed_sampleid = new_df.loc[new_df_imp.index, 'sample_id']

#imputing missing values by sampling from bottom 10% of values per protein for proteins not previously dealt with
for prot in new_df.columns[:-7]:
    if not new_df[prot].isnull().any(): 
        continue 
    quantile = new_df[prot].quantile(0.05)
    min_val = new_df[new_df[prot]<quantile][prot].min()
    max_val = new_df[new_df[prot]<quantile][prot].max()
    for idx in new_df_imp[new_df_imp[prot].isnull()][prot].index:
        sample = random.uniform(min_val, max_val) #bottom 10% 
        #sample = 0
        new_df_imp.at[idx,prot] = sample
        imputeddf.at[idx,prot] = sample

In [9]:
# make a df with NAs imputed with zeros, to show how much/what is getting imputed
new_df_zimp = new_df.copy() #copy it 

for prot in new_df.columns[0:-7]:
    for idx in new_df[new_df[prot].isnull()][prot].index:
        new_df_zimp.at[idx,prot] = 0

In [10]:
# remove problematic bits. columns with unexpected strings + NAs
tmp = new_df_imp.isna().sum()
#imp_clean =  new_df_imp.drop(tmp[tmp>0].index, axis = 1).drop(columns=['sample_id', 'Unnamed: 0'])
imp_clean =  new_df_imp.drop(tmp[tmp>0].index, axis = 1).drop(columns=['sample_id'])

In [16]:
#creating a heatmap and clustering by proteins and samples to see if there are trends associated with sample type, sex or GA weeks
#lut = {'EOS': sns.color_palette("Set3")[3], 'Control': sns.color_palette("Set3")[4], 'PS': 'g'}
#lut = {'EOS': sns.color_palette("husl", 8)[0], 'Control': sns.color_palette("husl", 8)[5], 'PS': 'g'}
lut = {'EOS': '#940043', 'Control': '#577483', 'PS': '#577483', 'PS_nonclassicEOS': '#577483'}
fourcol= sns.cubehelix_palette(start=3, rot=-.5, as_cmap=False, n_colors=4) #start =0.5 for pleasant blue, 
lut_ga = {'<28': fourcol[0], '28- <32': fourcol[1], '32-36': fourcol[2], '>=37': fourcol[3], None:'lightpink'}

lut_sex = {'Female': 'white', 'Male': 'black', None:'lightpink'}

row_colors_sample = imp_clean['sample_type'].map(lut)
row_colors_ga = imp_clean['ga_cat'].map(lut_ga)
row_colors_sex = imp_clean['sex'].map(lut_sex)
sns.set(font_scale=2)
testt = sns.cubehelix_palette(start =-0.3, rot=0.5, as_cmap=True, reverse=False)#0.3 for nice purp

sns.clustermap(imp_clean.iloc[:,0:-5], cmap = testt, figsize = [30,70],yticklabels = True,col_cluster = True, row_colors=[row_colors_sample,pd.Series(row_colors_sex).fillna('dimgray'),pd.Series(row_colors_ga).fillna('dimgray')])
#plt.savefig('graphs/fig1_heatmap_imputed.eps', format = 'eps',bbox_inches = 'tight')
plt.show()
plt.close()

ValueError: Invalid RGBA argument: nan

<Figure size 3000x7000 with 0 Axes>